## Partie 1 : PageRank - version itérative, premier exemple

##### 1. Justifier pourquoi l’algorithme de la puissance iterérée permet de calculer le score de chacune des pages. 

Dans un premier temps il faut que la matrice symbolysant le web soit une matrice stochastique, c'est une matrice de probabilité ou la somme de chaque colonne doit être égal à 1, elle traduit la probabilité de se rendre sur une autre page depuis la page courante. En appliquant l'algorythme de puissance itértée sur celle-ci on obtient des vecteurs propre pour chaque point de la matrice. Ces vecteurs nous donne la probabilité de se retrouvé sur une page apres tant d'itération. Plus sa valeur est grande plus la page est fréquemment visité au cour des itérations.    



In [8]:
from random import randint
import numpy as np
from math import sqrt

In [9]:
#fonction pour la norme
def norme(X:np.array):
    sum = 0 
    for x in X:
        sum += x**2
    return np.sqrt(sum)

V = np.array([2,0,0,3])
print(norme(V))

3.605551275463989


In [20]:

def puissanceItere(l,A:np.array):
   """Fonction qui applique l'algorithme de puissance itérée sur une matrice. 
   Matrice de toutes tailles, stochastique ou non"""
   X = np.array([randint(1,10) for x in range (A.shape[0])])
   Lambda = 0
   while (True):
       aLambda = Lambda
       AX = np.dot(A,X) # approximation du vecteur propre associé
       Lambda = norme(AX) # approximation de lambda
       X = AX / Lambda
       if (abs(Lambda - aLambda) < l ):
           return X,Lambda


C2 = np.array([[2, 3], [1, 0]])
X, Lambda = puissanceItere(1e-6, C2)
print("Vecteur propre approximé :", X)
print("Valeur propre approximée :", Lambda)
print("--------------------------------------")
C3 = np.array([[4,1,0], [1,3,1], [0,1,2]])
X, Lambda = puissanceItere(1e-6, C3)
print("Vecteur propre approximé :", X)
print("Valeur propre approximée :", Lambda)
print("--------------------------------------")
C4 = np.array([[4, 1, 0, 0],[1, 3, 1, 0],[0, 1, 2, 1],[0, 0, 1, 1]])
X, Lambda = puissanceItere(1e-6, C4)
print("Vecteur propre approximé :", X)
print("Valeur propre approximée :", Lambda)

Vecteur propre approximé : [0.94868329 0.31622778]
Valeur propre approximée : 2.9999999123875525
--------------------------------------
Vecteur propre approximé : [0.78886698 0.57715835 0.21113297]
Valeur propre approximée : 4.732050418674707
--------------------------------------
Vecteur propre approximé : [0.77773019 0.57997314 0.2342019  0.06258108]
Valeur propre approximée : 4.7452807772489995
